## Import documents exported from Evernote to a vectorstore
### Use OpenAI file search with responses API
#### Prerequisite steps
* exported notes from your Evernote notebook as html 
* converted the notes further to md-files and remove broken image links (use python/AI)
* the files are named with note titles

Files are in one folder.


##### Query ChromaDB vectorstore
I tried to accomplish this task with RAG like the example by https://github.com/ed-donner/llm_engineering/commits?author=dinorrusso.

I thought this to be a trivial task, but it was not 😃 That example uses Ollama running locally.
Even though the retriever had the information required, it was dropped from the answer.

I tried then to use Chroma + OpenAI. After several attemps succeeded to create a vectorstore and query it. That's it for this time.

##### Openai vectorstore, see bottom of the notebook
One attempt was to use OpenAI's fileSearch-tool which seemed pretty straightforward.
The con: loading files was not working always. Code is left though as reference.

In [0]:
#Imports
from dotenv import load_dotenv
import gradio as gr
import openai
import chromadb
from chromadb.config import Settings
import os

#### Load files to vectorstore

In [0]:

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
openai.api_key = os.environ['OPENAI_API_KEY']

def chunk_text(text, max_tokens=2000):
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        current_length += len(word) + 1  # +1 for the space
        if current_length > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_length = len(word) + 1
        else:
            current_chunk.append(word)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


# # Set up OpenAI API key
# openai.api_key = "your_openai_api_key"  # Replace with your API key
chroma_client = chromadb.Client()

# Create or get the existing collection
collection_name = "EverNotes"

try:
    existing_collection = chroma_client.get_collection(name=collection_name)
    if existing_collection.count() > 0:
        chroma_client.delete_collection(name=collection_name)
except:
    print(f"Collection {collection_name} does not exist. Creating a new one.")

# Create a collection in ChromaDB
collection = chroma_client.get_or_create_collection(name=collection_name)

# Define your data
# it should be like this
# documents = ["OpenAI is revolutionizing AI.", "ChromaDB makes embedding storage easy."]
# metadata = [{"id": 1}, {"id": 2}]

folder_path = os.getenv('EVERNOTE_EXPORT')
documents = []

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.md'):  # Change this to the file extension you need
            with open(os.path.join(root, file), 'r') as f:
                documents.append(f.read())

metadata = [{"id": i + 1} for i in range(len(documents))]

# Generate embeddings using OpenAI
def get_embedding(text, model="text-embedding-ada-002"):
    response = openai.embeddings.create(input=text, model=model)
    return response.data[0].embedding

# Add documents and embeddings to ChromaDB in chunks
for doc, meta in zip(documents, metadata):
    chunks = chunk_text(doc)
    for chunk in chunks:
        embedding = get_embedding(chunk)
        collection.add(
            documents=[chunk],
            embeddings=[embedding],
            metadatas=[meta],
            ids=[str(meta["id"])]
        )


#### Query ChromaDB

In [0]:
# 
query_text = "Is there a video for Fitting the Shimano speed hub 7"
query_embedding = get_embedding(query_text)

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=2
)

print("Query Results:", results)

##### Gradio interface

In [0]:
# Function to query ChromaDB
def query_chromadb(query_text):
    query_embedding = get_embedding(query_text)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=2
    )
    return results

# Gradio interface
def gradio_interface(query_text):
    results = query_chromadb(query_text)
    return results

# Create Gradio app
iface = gr.Interface(
    fn=gradio_interface,
    inputs="text",
    outputs="text",
    title="ChromaDB Query Interface",
    description="Enter your query to search the ChromaDB collection."
)

iface.launch()

#### Below OpenAI filesearch variant which had some failures in file uploads.

In [0]:
import glob
folder_path = os.environ['EVERNOTE_EXPORT'] 
# Filter out other except .md-files
md_files = glob.glob(os.path.join(folder_path, '*.md'))
file_paths = [os.path.join(folder_path, file) for file in md_files]
file_streams = [open(path, 'rb') for path in file_paths]

# Create vector store
vector_store = openai.vector_stores.create(
    name="Evernote notes",
)

# Batch Upload Limit: You can upload up to 100 files in a single batch
# https://community.openai.com/t/max-100-files-in-vector-store/729876/4
batch_size = 90
for i in range(0, len(file_streams), batch_size):
    batch = file_streams[i:i + batch_size]
    file_batch = openai.vector_stores.file_batches.upload_and_poll(
        vector_store_id=vector_store.id,
        files=batch
    )
    print(file_batch.status)
    print(file_batch.file_counts)

# There can be some fails in file counts:
# "FileCounts(cancelled=0, completed=89, failed=1, in_progress=0, total=90)""
# Usually 1 % fails. Did not find solution for improving that yet

In [0]:


response = openai.responses.create(
    model="gpt-4o-mini",
    input="Is there a video for Fitting the Shimano speed hub 7?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
    }],
    include=None
)
print(response)